In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import os

# proxyの設定．

# torchvision.datasetsを利用する場合は，datasetを直接ダウンロードするので，学内マシンからは通常必要．
os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"      # "0":GPU0, "1":GPU1, "0,1":GPUを2つとも使用

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
!wget www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

--2020-05-19 20:23:47--  http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving proxy.uec.ac.jp (proxy.uec.ac.jp)... 130.153.8.24
Connecting to proxy.uec.ac.jp (proxy.uec.ac.jp)|130.153.8.24|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: '20170201.tar.bz2'

100%[====================================>] 1,373,795,477 11.2MB/s   in 1m 59s 

2020-05-19 20:25:45 (11.0 MB/s) - '20170201.tar.bz2' saved [1373795477/1373795477]



In [ ]:
import sys
!{sys.executable} -m pip install --user gensim

In [18]:
import gensim
model_dir = 'entity_vector/entity_vector.model.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [22]:
#単語ベクトル
vector = model.wv['電気通信大学'] 
vector

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([-0.09985781, -0.04528721,  0.14309461,  0.04331237, -0.05111622,
        0.02780907, -0.0164357 , -0.03563423,  0.06376494, -0.06248141,
       -0.11966642,  0.14175622, -0.02436817, -0.15606536, -0.00073292,
        0.03640464, -0.10770931,  0.01223423,  0.06853946, -0.05768216,
        0.03636627, -0.05685209,  0.02971395, -0.13778223,  0.04839297,
       -0.18639874, -0.08707647,  0.03588375, -0.02607614, -0.09608255,
       -0.072992  , -0.00875725, -0.05510923,  0.07588227,  0.20048644,
        0.03430462,  0.11711261,  0.12323629, -0.06671874, -0.17651275,
       -0.14487796,  0.0780989 , -0.11762877,  0.03193246, -0.09853007,
       -0.0228832 , -0.0319857 ,  0.05298912, -0.10750021, -0.03470328,
        0.06585295, -0.09762403,  0.08115529,  0.00505495, -0.03916613,
       -0.04675321,  0.07809863, -0.07920853,  0.17390606,  0.03019071,
        0.12526204, -0.00118598, -0.00435806, -0.13399956, -0.02860726,
       -0.01012302,  0.04259544,  0.12178548, -0.00323822,  0.03

In [37]:
#類似単語
results = model.most_similar(u'[ピカチュウ]')
for result in results:
    print(result)

('ポケモン', 0.8059247732162476)
('[ピカチュウ_(サトシのポケモン)]', 0.7879241704940796)
('[カービィ]', 0.7576284408569336)
('[ポケットモンスター_(架空の生物)]', 0.748759388923645)
('[ワリオ]', 0.7456034421920776)
('[ヨッシー]', 0.7434700727462769)
('ピカチュウ', 0.7234353423118591)
('[ヒカリ_(アニメポケットモンスター)]', 0.721408486366272)
('[ミュウツー]', 0.7161428928375244)
('[マリオ_(ゲームキャラクター)]', 0.7144089341163635)


In [42]:
#引き算
results = model.most_similar(positive=[u'[大学]'],negative=[u'[数学]'])
for result in results:
    print(result)

('[大学生協]', 0.47519373893737793)
('生協', 0.46215200424194336)
('[町内会]', 0.45809900760650635)
('私立大学', 0.4549909234046936)
('[私立大学]', 0.45075586438179016)
('自治会', 0.4495047926902771)
('[医科大学]', 0.44734373688697815)
('[公立大学]', 0.4456261396408081)
('[日本人学校]', 0.44493407011032104)
('[女子大学]', 0.4444669485092163)


In [52]:
#足し算
results = model.most_similar(positive=[u'[東京]',u'[公園]'])
for result in results:
    print(result)

('[日比谷公園]', 0.7260562181472778)
('[上野恩賜公園]', 0.7071521282196045)
('[青山_(東京都港区)]', 0.6866978406906128)
('[月島]', 0.6848454475402832)
('[赤坂_(東京都港区)]', 0.6832717657089233)
('[芝公園]', 0.6748826503753662)
('[代々木公園]', 0.6744815111160278)
('[上野]', 0.6725881099700928)
('[新宿御苑]', 0.669266939163208)
('[北の丸公園]', 0.6659857034683228)


In [81]:
#加減算
results = model.most_similar(positive=[u'[東京大学]',u'[京都]'],negative=[u'[東京]'])
for result in results:
    print(result)

('[京都大学]', 0.6379557847976685)
('[京都帝国大学]', 0.6370033621788025)
('[大谷大学]', 0.6262593865394592)
('東大', 0.6203417778015137)
('[東京帝国大学]', 0.6098219156265259)
('[國學院大學]', 0.6031330823898315)
('[東北大学]', 0.6001753807067871)
('[九州大学]', 0.5959972143173218)
('東京大学', 0.5811233520507812)
('[大正大学]', 0.5761975049972534)


In [108]:
results = model.most_similar(positive=[u'[博物館]',u'[東京]'])
for result in results:
    print(result)

('[美術館]', 0.6756891012191772)
('[日比谷公園]', 0.6479947566986084)
('[上野恩賜公園]', 0.6453565359115601)
('[科学館]', 0.642791211605072)
('[国立]', 0.6348460912704468)
('書籍館', 0.6338818669319153)
('[横浜]', 0.6328469514846802)
('[赤坂_(東京都港区)]', 0.6288894414901733)
('[文学館]', 0.6271750926971436)
('[青山_(東京都港区)]', 0.6255673170089722)


In [96]:
results = model.most_similar(positive=[u'[東京]',u'[アメリカ]'],negative=[u'[日本]'])
for result in results:
    print(result)

('[ニューヨーク]', 0.7211385369300842)
('[シカゴ]', 0.6864314079284668)
('[サンフランシスコ]', 0.6799116730690002)
('ニューヨーク', 0.6757162809371948)
('[ロサンゼルス]', 0.6688859462738037)
('ロサンゼルス', 0.6533620357513428)
('サンフランシスコ', 0.6382411122322083)
('[ロンドン]', 0.6338860988616943)
('[シアトル]', 0.6219587326049805)
('[トロント]', 0.6208849549293518)


In [101]:
results = model.most_similar(positive=[u'[池袋]',u'[大阪]'],negative=[u'[東京]'])
for result in results:
    print(result)

('[心斎橋]', 0.7890547513961792)
('[日本橋_(大阪市)]', 0.7753708362579346)
('[千日前]', 0.765419602394104)
('[下北沢]', 0.7421362996101379)
('[梅田]', 0.7394716739654541)
('[新世界_(大阪)]', 0.7393956780433655)
('[有楽町]', 0.7377620935440063)
('[道頓堀]', 0.7368851900100708)
('[北新地]', 0.7365112900733948)
('池袋', 0.7348434925079346)
